# WeatherPy

---

## Starter Code to Generate Random Geographic Coordinates and a List of Cities

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

# Impor the OpenWeatherMap API key
from config import weather_api_key

# Import citipy to determine the cities based on latitude and longitude
from citipy import citipy

### Generate the Cities List by Using the `citipy` Library

In [2]:
# Empty list for holding the latitude and longitude combinations
lat_lngs = []

# Empty list for holding the cities names
cities = []

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

for i in range(len(cities)):
    cities[i] = cities[i].title()
    
# Print the city count to confirm sufficient count
print(f"Number of cities in the list: {len(cities)}")



Number of cities in the list: 601
['Tessalit', 'Saint-Pierre', 'Rikitea', 'Izhma', 'Atuona', 'Trelew', 'Okitipupa', 'Maceio', 'Porto Velho', 'Palu', 'Yellowknife', 'Dzhusaly', 'Itacoatiara', 'Palabuhanratu', 'Arraial Do Cabo', 'Inuvik', 'Margate', 'Nanortalik', 'Christchurch', 'Hilo', 'Ushuaia', 'Qaanaaq', 'Vanimo', 'Castro', 'Jamestown', 'San Cristobal', 'Povorino', 'Mar Del Plata', 'Ahipara', 'Saint-Francois', 'Cabo San Lucas', 'Taolanaro', 'Severo-Kurilsk', 'Minnehaha', 'Hearst', 'Punta Arenas', 'Belushya Guba', 'Saint-Philippe', 'Talnakh', 'Nikolskoye', 'Ribeira Do Pombal', 'Pevek', 'Ponta Delgada', 'Kapaa', 'Albany', 'Puerto Ayora', 'Bredasdorp', 'Pisco', 'Opuwo', 'Mahina', 'Sandwick', 'Provideniya', 'Whitehorse', 'Geraldton', 'Beyneu', 'Busselton', 'Mataura', 'Sentyabrskiy', 'Caravelas', 'Dikson', 'Kaitangata', 'Zhaotong', 'Tiksi', 'Saldanha', 'Vaini', 'Vila Franca Do Campo', 'Moshi', 'Ereymentau', 'Port Lincoln', 'Katsuura', 'Lorengau', 'Muros', 'Atbasar', 'Santa Vitoria Do Palm

---

## Requirement 1: Create Plots to Showcase the Relationship Between Weather Variables and Latitude

### Use the OpenWeatherMap API to retrieve weather data from the cities list generated in the started code

In [12]:
# Set the API base URL\
url = "http://api.openweathermap.org/data/2.5/weather?q="

# Define an empty list to fetch the weather data for each city
city_data = []

# Print to logger
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters
record_count = 1
set_count = 1

# Loop through all the cities in our list to fetch weather data
for i , city in enumerate(cities):
        
    # Group cities in sets of 50 for logging purposes
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 0

    # Create endpoint URL with each city
    city_url = f"{url}{cities[i]}&appid={weather_api_key}"
    
    # Log the url, record, and set numbers
    print("Processing Record %s of Set %s | %s" % (record_count, set_count, city))

    # Add 1 to the record count
    record_count += 1

    # Run an API request for each of the cities
    try:
        response = requests.get(city_url).json() # Parse the JSON and retrieve data

        city_weather = response["weather"][0]["main"]
        # Parse out latitude, longitude, max temp, humidity, cloudiness, wind speed, country, and date

        city_lat = response["coord"]["lat"]  
        city_lng = response["coord"]["lon"]         
        city_max_temp = (response["main"]["temp_max"] - 273) 
        city_humidity = response["main"]["humidity"]    
        city_clouds = response["clouds"]["all"]    
        city_wind = response["wind"]["speed"]       
        city_country = response["sys"]["country"]
        city_date = response["dt"] 

        # Append the City information into city_data list
        city_data.append({"City": city, 
                            "Lat": city_lat, 
                            "Lng": city_lng, 
                            "Max Temp": city_max_temp,
                            "Humidity": city_humidity,
                            "Cloudiness": city_clouds,
                            "Wind Speed": city_wind,
                            "Country": city_country,
                            "Date": city_date
                            })

    # If an error is experienced, skip the city
    except:
        print("City not found. Skipping...")
        pass
              
# Indicate that Data Loading is complete 
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | Tessalit
Processing Record 2 of Set 1 | Saint-Pierre
Processing Record 3 of Set 1 | Rikitea
Processing Record 4 of Set 1 | Izhma
Processing Record 5 of Set 1 | Atuona
Processing Record 6 of Set 1 | Trelew
Processing Record 7 of Set 1 | Okitipupa
Processing Record 8 of Set 1 | Maceio
Processing Record 9 of Set 1 | Porto Velho
Processing Record 10 of Set 1 | Palu
Processing Record 11 of Set 1 | Yellowknife
Processing Record 12 of Set 1 | Dzhusaly
City not found. Skipping...
Processing Record 13 of Set 1 | Itacoatiara
Processing Record 14 of Set 1 | Palabuhanratu
City not found. Skipping...
Processing Record 15 of Set 1 | Arraial Do Cabo
Processing Record 16 of Set 1 | Inuvik
Processing Record 17 of Set 1 | Margate
Processing Record 18 of Set 1 | Nanortalik
Processing Record 19 of Set 1 | Christchurch
Processing Record 20 of Set 1 | Hilo
Processing Record 21 of Set 1 | Ushuaia
Processing Record 22 o

In [13]:
# Convert the cities weather data into a Pandas DataFrame
city_data_df = pd.DataFrame(city_data)

# Show Record Count
city_data_df.count()

City          553
Lat           553
Lng           553
Max Temp      553
Humidity      553
Cloudiness    553
Wind Speed    553
Country       553
Date          553
dtype: int64

In [17]:
# Display sample data
city_data_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Tessalit,20.1986,1.0114,23.03,13,74,4.28,ML,1674034443
1,Saint-Pierre,-21.3393,55.4781,31.21,70,75,15.43,RE,1674034444
2,Rikitea,-23.1203,-134.9692,25.64,71,25,1.91,PF,1674034445
3,Izhma,65.0083,53.9113,-11.43,95,66,3.06,RU,1674034445
4,Atuona,-9.8000,-139.0333,25.44,80,26,9.15,PF,1674034446


In [18]:
# Export the City_Data into a csv
city_data_df.to_csv("output_data/cities.csv", index_label="City_ID")

In [21]:
# Read saved data
city_data_df = pd.read_csv("output_data/cities.csv", index_col="City_ID")

# Display sample data
city_data_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
City_ID,,,,,,,,,
0,Tessalit,20.1986,1.0114,23.03,13,74,4.28,ML,1674034443
1,Saint-Pierre,-21.3393,55.4781,31.21,70,75,15.43,RE,1674034444
2,Rikitea,-23.1203,-134.9692,25.64,71,25,1.91,PF,1674034445
3,Izhma,65.0083,53.9113,-11.43,95,66,3.06,RU,1674034445
4,Atuona,-9.8000,-139.0333,25.44,80,26,9.15,PF,1674034446


In [ ]:
#Latitude vs. Temperature

In [ ]:
#Latitude vs. Humidity

In [ ]:
#Latitude vs. Cloudiness

In [ ]:
#Latitude vs. Wind Speed

### Create the Scatter Plots Requested

#### Latitude Vs. Temperature

In [ ]:
# Build scatter plot for latitude vs. temperature
# YOUR CODE HERE

# Incorporate the other graph properties
# YOUR CODE HERE

# Save the figure
plt.savefig("output_data/Fig1.png")

# Show plot
plt.show()

#### Latitude Vs. Humidity

In [ ]:
# Build the scatter plots for latitude vs. humidity
# YOUR CODE HERE

# Incorporate the other graph properties
# YOUR CODE HERE

# Save the figure
plt.savefig("output_data/Fig2.png")

# Show plot
plt.show()

#### Latitude Vs. Cloudiness

In [ ]:
# Build the scatter plots for latitude vs. cloudiness
# YOUR CODE HERE

# Incorporate the other graph properties
# YOUR CODE HERE

# Save the figure
plt.savefig("output_data/Fig3.png")

# Show plot
plt.show()

#### Latitude vs. Wind Speed Plot

In [ ]:
# Build the scatter plots for latitude vs. wind speed
# YOUR CODE HERE

# Incorporate the other graph properties
# YOUR CODE HERE

# Save the figure
plt.savefig("output_data/Fig4.png")

# Show plot
plt.show()

---

## Requirement 2: Compute Linear Regression for Each Relationship


In [ ]:
# Define a function to create Linear Regression plots
# YOUR CODE HERE

In [ ]:
# Create a DataFrame with the Northern Hemisphere data (Latitude >= 0)
# YOUR CODE HERE

# Display sample data
northern_hemi_df.head()

In [ ]:
# Create a DataFrame with the Southern Hemisphere data (Latitude < 0)
# YOUR CODE HERE

# Display sample data
southern_hemi_df.head()

###  Temperature vs. Latitude Linear Regression Plot

In [ ]:
# Linear regression on Northern Hemisphere
# YOUR CODE HERE

In [ ]:
# Linear regression on Southern Hemisphere
# YOUR CODE HERE

**Discussion about the linear relationship:** YOUR RESPONSE HERE

### Humidity vs. Latitude Linear Regression Plot

In [ ]:
# Northern Hemisphere
# YOUR CODE HERE

In [ ]:
# Southern Hemisphere
# YOUR CODE HERE

**Discussion about the linear relationship:** YOUR RESPONSE HERE

### Cloudiness vs. Latitude Linear Regression Plot

In [ ]:
# Northern Hemisphere
# YOUR CODE HERE

In [ ]:
# Southern Hemisphere
# YOUR CODE HERE

**Discussion about the linear relationship:** YOUR RESPONSE HERE

### Wind Speed vs. Latitude Linear Regression Plot

In [ ]:
# Northern Hemisphere
# YOUR CODE HERE

In [ ]:
# Southern Hemisphere
# YOUR CODE HERE

**Discussion about the linear relationship:** YOUR RESPONSE HERE